# Hugonnet DEM void imputation (RBF sampler + Bayesian ridge)


This notebook implements the following logic:

- Build/read a library of connected void-pattern masks
- Compute per-glacier predictor stacks (x/y, Cop30 elevation, slope, aspect sin/cos, year)
- Create synthetic voids by placing library patterns onto non-void areas until a target proportion is reached
- Tune hyperparameters (gamma, n_components) with Bayesian optimisation using 5 synthetic realisations
- Fit final per-glacier models using the selected hyperparameters
- Predict full rasters per year and overwrite the original voids

## Imports and configuration

In [1]:
import numpy as np
import pandas as pandas
import xarray
import rioxarray
from scipy import ndimage

import sklearn.pipeline
import sklearn.kernel_approximation
import sklearn.linear_model

import skopt
import skopt.space
import skopt.utils
import skopt.callbacks

from tqdm import tqdm
import os
import glob
import pickle

## Paths (edit as needed)


Make sure that Cop30DEM elevation and derivatives are reprojected to the same grid as the Hugonnet et al. data.

In [4]:
COP30_ELEVATION_DIR = "cop30dem/elevation"
HUGONNET_TILES_DIR = "hugonnet_dems/tiles"

COP30_SLOPE_DIR = "cop30dem/slope"
COP30_ASPECT_DIR = "cop30dem/aspect"

OUT_IMPUTING_MODELS_DIR = "outputs/imputing_models"
OUT_HP_TUNING_DIR = "outputs/hp_tuning"
OUT_MODELLED_DIR = "outputs/modelled_by_rbf_br"

os.makedirs(OUT_IMPUTING_MODELS_DIR, exist_ok=True)
os.makedirs(OUT_HP_TUNING_DIR, exist_ok=True)
os.makedirs(OUT_MODELLED_DIR, exist_ok=True)

## Helper functions

In [5]:
def list_glaciers():
    tif_paths = glob.glob(os.path.join(COP30_ELEVATION_DIR, "*.tif"))
    glaciers = sorted([os.path.basename(p)[:-4] for p in tif_paths])
    return glaciers


def load_hugonnet_tile_paths(glacier):
    h_elev_paths = sorted(glob.glob(os.path.join(HUGONNET_TILES_DIR, f"Z_{glacier}_*.tif")))
    h_stddev_paths = sorted(glob.glob(os.path.join(HUGONNET_TILES_DIR, f"ZSTDDEV_{glacier}_*.tif")))
    return h_elev_paths, h_stddev_paths


def compute_original_void_mask(h_elev_paths):
    stack = []
    for elev_path in h_elev_paths:
        elev = rioxarray.open_rasterio(elev_path, masked=True)[0]
        stack.append(elev.data)
    stack = np.stack(stack, axis=0)
    voids = np.isnan(stack).any(axis=0).astype(np.uint8)
    return voids


def extract_connected_void_patterns(void_mask):
    if np.all(void_mask == 0):
        return []

    labelled, _ = ndimage.label(void_mask, structure=[[1, 1, 1], [1, 1, 1], [1, 1, 1]])
    slices = ndimage.find_objects(labelled)

    patterns = []
    for lbl, slc in enumerate(slices, start=1):
        void = (labelled[slc] == lbl).astype(np.uint8)
        patterns.append(void)
    return patterns


def load_cop30_predictors(glacier):
    cop30_elev = rioxarray.open_rasterio(os.path.join(COP30_ELEVATION_DIR, f"{glacier}.tif"), masked=True)[0].rio.interpolate_na()
    cop30_slope = rioxarray.open_rasterio(os.path.join(COP30_SLOPE_DIR, f"{glacier}.tif"), masked=True)[0].rio.interpolate_na()
    cop30_aspect = rioxarray.open_rasterio(os.path.join(COP30_ASPECT_DIR, f"{glacier}.tif"), masked=True)[0].rio.interpolate_na()

    cop30_aspect = cop30_aspect / 180.0 * np.pi
    aspect_sin = np.sin(cop30_aspect.data)
    aspect_cos = np.cos(cop30_aspect.data)

    xv = cop30_elev["x"].values
    yv = cop30_elev["y"].values
    xgrid, ygrid = np.meshgrid(xv, yv)
    x_norm = (xgrid - xv.min()) / (xv.max() - xv.min())
    y_norm = (ygrid - yv.min()) / (yv.max() - yv.min())

    return x_norm, y_norm, cop30_elev.data, cop30_slope.data, aspect_sin, aspect_cos


def build_xyw_triples_for_glacier(glacier, h_elev_paths, h_stddev_paths):
    """
    For each year tile, assemble:
      predictors: [x_norm, y_norm, cop30_elev, cop30_slope, aspect_sin, aspect_cos, year]
      target: (hugonnet_z - cop30_elev)
      alpha: zstddev^2  (used later as inverse-variance-ish weights)
    """
    x_norm, y_norm, cop30_elev, cop30_slope, aspect_sin, aspect_cos = load_cop30_predictors(glacier)

    xyws = []
    for z_path, zstddev_path in zip(h_elev_paths, h_stddev_paths):
        year = float(os.path.basename(z_path)[:-4].split("_")[-1])

        z = rioxarray.open_rasterio(z_path, masked=True)[0]
        zstddev = rioxarray.open_rasterio(zstddev_path, masked=True)[0]

        alpha = (zstddev ** 2).data
        target = (z.data - cop30_elev)

        years = np.full_like(cop30_elev, year, dtype=float)

        predictors = [
            x_norm,
            y_norm,
            cop30_elev,
            cop30_slope,
            aspect_sin,
            aspect_cos,
            years,
        ] # 7 predictors
        predictors = np.stack(predictors, axis=-1)
        xyws.append((predictors, target, alpha))

    return xyws


def compute_predictor_minmax(xyw_triples):
    predictors_mins = np.full(7, np.inf) # 7 predictors hard-coded here
    predictors_maxs = np.full(7, -np.inf)

    for x, _, _ in xyw_triples:
        loc_mins = np.min(x, axis=(0, 1))
        loc_maxs = np.max(x, axis=(0, 1))
        predictors_mins = np.minimum(predictors_mins, loc_mins)
        predictors_maxs = np.maximum(predictors_maxs, loc_maxs)

    return predictors_mins, predictors_maxs